# Sports 1M C3D Network to Keras

In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
from keras.optimizers import SGD

Using TensorFlow backend.


In [12]:
def get_model(summary=False):
    """ Return the Keras model of the network
    """
    model = Sequential()
    # 1st layer group
    model.add(Convolution3D(64, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv1',
                            subsample=(1, 1, 1), 
                            input_shape=(3, 16, 112, 112)))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), 
                           border_mode='valid', name='pool1'))
    # 2nd layer group
    model.add(Convolution3D(128, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv2',
                            subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           border_mode='valid', name='pool2'))
    # 3rd layer group
    model.add(Convolution3D(256, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv3a',
                            subsample=(1, 1, 1)))
    model.add(Convolution3D(256, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv3b',
                            subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(2, 2, 2), 
                           border_mode='valid', name='pool3'))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    # 4th layer group
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv4a',
                            subsample=(1, 1, 1)))
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv4b',
                            subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(2, 2, 2), 
                           border_mode='valid', name='pool4'))
    # 5th layer group
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv5a',
                            subsample=(1, 1, 1)))
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv5b',
                            subsample=(1, 1, 1)))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(2, 2, 2), 
                           border_mode='valid', name='pool5'))
    model.add(Flatten())
    # FC layers group
    model.add(Dense(2048, activation='relu', name='fc6'))
    model.add(Dropout(.5))
    model.add(Dense(2048, activation='relu', name='fc7'))
    model.add(Dropout(.5))
    model.add(Dense(487, activation='softmax', name='fc8'))
    if summary:
        print(model.summary())
    return model

model = get_model(summary=True)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
conv1 (Convolution3D)            (None, 3, 16, 112, 64 193600      convolution3d_input_11[0][0]     
____________________________________________________________________________________________________
pool1 (MaxPooling3D)             (None, 3, 8, 56, 64)  0           conv1[0][0]                      
____________________________________________________________________________________________________
conv2 (Convolution3D)            (None, 3, 8, 56, 128) 221312      pool1[0][0]                      
____________________________________________________________________________________________________
pool2 (MaxPooling3D)             (None, 1, 4, 28, 128) 0           conv2[0][0]                      
___________________________________________________________________________________________

# Loding Paramater

In [ ]:
import caffe_pb2 as caffe
import numpy as np

p = caffe.NetParameter()
p.ParseFromString(
    open('model/conv3d_deepnetA_sport1m_iter_1900000', 'rb').read()
)

def rot90(W):
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            for k in range(W.shape[2]):
                W[i, j, k] = np.rot90(W[i, j, k], 2)
    return W

params = []
conv_layers_indx = [1, 4, 7, 9, 12, 14, 17, 19]
fc_layers_indx = [22, 25, 28]

for i in conv_layers_indx:
    layer = p.layers[i]
    weights_b = np.array(layer.blobs[1].data, dtype=np.float32)
    weights_p = np.array(layer.blobs[0].data, dtype=np.float32).reshape(
        layer.blobs[0].num, layer.blobs[0].channels, layer.blobs[0].length,
        layer.blobs[0].heights, layer.blobs[0].width
    )
    weithts_p = rot90(weights_p)
    params.append([weights_p, weights_b])
    
for i in fc_layers_indx:
    layer = p.layers[i]
    weights_b = np.array(layer.blobs[1].data, dtype=np.float32)
    weights_p = np.array(layer.blobs[0].data, dtype=np.float32).reshape(
        layer.blobs[0].num, layer.blobs[0].channels, layer.blobs[0].length,
        layer.blobs[0].length, layer.blobs[0].width)[0,0,0,:,:].T
    params.append([weights_p, weights_b])

In [ ]:
Now that all the parameter loaded, let put it into the model

In [ ]:
model_layers_indx = [0, 2, 4, 5, 7, 8, 10, 11] + [15, 17, 19]
for i, j in zip(model_layers_indx, range(11)):
    model_layers[i].set_weights(params[j])

In [ ]:
Get the model weights

In [ ]:
import h5py

model.save_weghts("sports1M_weights.h5", overwrite=True)
json_string = model.to_json()
with open("sports1M_model.json", "w") as f:
    f.write(json_string)

In [ ]:
# Test

In [ ]:
from keras.models import model_from_json

model = model_from_json(open("sports1M_model.json", "r").read())
model.load_weights("sports1M_weights.h5")
model.compile(loss="mean_squared_error", optimizer="sgd")

In [ ]:
For testing, we are going to load the weight

In [ ]:
with open("./model/lebels.txt", "r") as f:
    labels = [line.strip() for line in f.readlines()]
print("Total labels: {}".format(len(label)))

In [ ]:
For testing we are going to load video from the Suports1M data sets and pass it through the model.

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture("dM06AMFLsrc.mp4")

vid = []
while True:
    ret, img = cap.read()
    if not ret:
        break
    vid.append(cv2.resize(img, (171, 128)))
vid = np.array(vid, dtype=np.float32)

In [ ]:
Plot a frame of the video

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(vid[2000]/256)

In [ ]:
Now form the video extra 16 fram clip and crop the center to get 3*16*112*112

In [ ]:
X = vid[2000:2016, 8:120, 30:142, :].transpose((3, 0, 1, 2))
output = model.predict_on_batch(np.array(X))
plt.plot(output[0][0])

In [ ]:
print("Position of maximum probability: {}".format(output[0].argmax()))
print("Maximum probability: {:.5f}".format(max(output[0][0])))
print("Corresponding label: {}".format(labels[output[0].argmax()]))

top_inds = output[0][0].argsort()[::-1][:5]
print("\nTop 5 probabilities and labels:")
_ = [print("{*.5f} {}".format(output[0][0][i], labels[i])) for i in top_inds]